In [1]:
from math import *
import numpy as np
from scipy.stats import entropy

<div align="center">
    <h1>Trabajo Práctico 2: Entropía</h1>
    <h2>Inferencia y Estimación</h2>
    <p>Alumna: Tahta Dadourian Martina</p>
    <p>Correo Electrónico: mtahtadadourian@udesa.edu.ar</p>
</div>

El siguiente trabajo práctico se centra en el estudio de la entropía de y entre variables, para definir la dependencia o independencia de variables aleatorias. Se utiliza un dataset de mediciones del ancho y largo de hojas de dos clases distintas, y se busca comprender la relación entre estas características dentro de una clase y entre clases. En ambos, se toman 222 muestras.

Además, se utiliza el libro *Elements of Information Theory*, escrito por Thomas M. Cover y Joy A. Thomas, como referencia para las definiciones de las ecuaciones y la teoría detrás.

In [2]:
def leer_csv(arch):
    """Lector de archivo con dataset de hojas. """
    
    clase1 = {'largo': [], 'ancho': []}
    clase2 = {'largo': [], 'ancho': []}

    with open(arch, 'r') as a:
        lines = a.readlines()
        for line in lines[1:]:
            line = line.rstrip()
            line = line.split(',')
            if line[0] == 'C1':
                clase1['largo'].append(float(line[1]))
                clase1['ancho'].append(float(line[2]))
            else:
                clase2['largo'].append(float(line[1]))
                clase2['ancho'].append(float(line[2]))

    return clase1, clase2

c1, c2 = leer_csv('dataset_hojas.csv')
size_c2 = len(c2['largo']) #Se achica el tamaño de las muestras de clase 1 para comparar a clase dos, porque tiene más muestras.
c1['largo'] = c1['largo'][:size_c2]
c1['ancho'] = c1['ancho'][:size_c2]

### Entropía individual
Comenzando con el cálculo de la entropía individual del ancho y largo de cada clase. El libro define a la entropía como una medida de la incerteza de una variable aleatoria. Se calcula la entropía de una variable aleatoria discreta a partir de la ecuación:
$$
H(X) = - \sum_{x \in X} p(x) \log_2{p(x)},
$$
siendo $X$ el conjunto de muestras y $p(x)$ la probabilidad de una muestra $x$. En el caso en que $p(x) = 0$, se cumple que $0\log_2{0} = 0$. Además, por su definición se comprende que $H(x) \geq 0$.

La entropía dice la cantidad de información requerida para describir la variable a la que se la calcula. En el caso de usar el logaritmo en base 2, la entropía dice la cantidad de bits necesarios para describirla.  

En el experimento, las entorpía son las siguientes:

In [3]:
def marg(x, anch):
    """Cálculo de marginal. """

    p_x, bin_edges = np.histogram(x, anch)
    p_x = p_x/len(x)
    return p_x, bin_edges[1]-bin_edges[0]

def h(p_x, anch):
    """Cálculo de entropía de una variable. """
    h = 0
    for p in p_x:
        h -= p*log2(p/anch) #/anch para que de la misma entropia para cualquier ancho de bins

    return h


print('---- Clase 1 ----')
marg_l1, anch_l1 = marg(c1['largo'], 5)
marg_a1, anch_a1 = marg(c1['ancho'], 5)
h_l1 = h(marg_l1, anch_l1)
h_a1 = h(marg_a1, anch_a1)
print(f'H(largo1) = {round(h_l1, 3)}')
print(f'H(ancho1) = {round(h_a1, 3)}')

print('\n---- Clase 2 ----')
marg_l2, anch_l2 = marg(c2['largo'], 5)
marg_a2, anch_a2 = marg(c2['ancho'], 5)
h_l2 = h(marg_l2, anch_l2)
h_a2 = h(marg_a2, anch_a2)
print(f'H(largo2) = {round(h_l2, 3)}')
print(f'H(ancho2) = {round(h_a2, 3)}')

---- Clase 1 ----
H(largo1) = 5.358
H(ancho1) = 4.511

---- Clase 2 ----
H(largo2) = 5.161
H(ancho2) = 4.224


### Entropía conjunta

De la misma manera, se puede pensar en un vector aleatorio y medir la entropía conjunta entre sus variables. En este caso, la ecuación es:
$$
H(X, Y) = - \sum_{x \in X}\sum_{y \in Y} p(x, y) \log_2{p(x, y)}
$$

In [4]:
def conjunta(m1, m2, bins):
    """Calcular la función conjunta (normalizada y original) de dos variables aleatorias con bins. """
    
    conj, _, _ = np.histogram2d(m1, m2, bins)
    conj_norm = conj/np.sum(conj)
    return conj, conj_norm


def h_conj(conj):
    """Calcular la entropía conjunta. """
    
    h = 0
    for f in conj:
        for j in f:
            if j!=0:
                h -= j*(log2(j))
    return h

conj_c1, conj_c1_norm = conjunta(c1['largo'], c1['ancho'], 5)
h_la_1 = h_conj(conj_c1_norm)
print('-------- Clase 1 --------')
print(f'H(largo1, ancho1) = {round(h_la_1, 3)}')

conj_c2, conj_c2_norm = conjunta(c2['largo'], c2['ancho'], 5)
h_la_2 = h_conj(conj_c2_norm)
print('\n-------- Clase 2 --------')
print(f'H(largo2, ancho2) = {round(h_la_2, 3)}')

-------- Clase 1 --------
H(largo1, ancho1) = 3.146

-------- Clase 2 --------
H(largo2, ancho2) = 2.854


### Entropía condicional
También, se define la entropía condicional entre dos varibles como:
$$
H(X|Y) = - \sum_{x \in X}\sum_{y \in Y} p(x, y) \log_2{p(y|x)}
$$


In [5]:
def h_cond(conj):
    """Cálculo de la entropía condicional. """
    
    p_A = np.sum(conj, axis=0)
    p_condi= conj/ p_A
    h= 0
    for x in range(len(p_condi)):
        for i in range(len(p_condi)):
                if conj[x][i] != 0 and p_condi[x][i] !=0:
                    h-= conj[x][i]*np.log2(p_condi[x][i])
    return h

print('------- Clase 1 --------')
h_cond_a_l_1 = h_cond(np.transpose(conj_c1_norm))
print(f'H(ancho1|largo1) = {round(h_cond_a_l_1, 3)}')

print('\n------- Clase 2 --------')
h_cond_a_l_2 = h_cond(np.transpose(conj_c2_norm))
print(f'H(ancho2|largo2) = {round(h_cond_a_l_2, 3)}')

print('\n------- Anchos ---------')
conj_ancho, conj_ancho_norm = conjunta(c1['ancho'], c2['ancho'], 5)
h_cond_a = h_cond(conj_ancho_norm)
print(f'H(ancho1|ancho2) = {round(h_cond_a, 3)}')

print('\n------- Largos ---------')
conj_largo, conj_largo_norm = conjunta(c1['largo'], c2['largo'], 5)
h_cond_l = h_cond(conj_largo_norm)
print(f'H(largo1|largo2) = {round(h_cond_l, 3)}')

------- Clase 1 --------
H(ancho1|largo1) = 1.248

------- Clase 2 --------
H(ancho2|largo2) = 1.125

------- Anchos ---------
H(ancho1|ancho2) = 1.847

------- Largos ---------
H(largo1|largo2) = 1.852


Con la entropía condicional y la individual, se puede definir la conjunta de la siguiente manera:
$$
H(X, Y) = H(X) + H(Y|X)
$$

Además, si se cumple que son independientes, la condicional $H(Y|X) = H(Y)$, por lo que $H(X, Y) = H(X) + H(Y)$. Por la definición de entropía (la cantidad de bits necesarios para describir una variable), mayor es la diferencia entre la entropía condicional y la individual, mayor es el aporte de la variable que condiciona a la descripición de la variable condicionada.

Se prueba esta relación entre las muestras, primero comparando el ancho con el largo de una clase, para ver si son independientes estas características en una hoja. 

In [6]:
print('------- Clase 1 --------')
h_cond_l_a_1 = h_cond(conj_c1_norm)
print(f'H(ancho1|largo1) = {round(h_cond_a_l_1, 3)}')
print(f'H(ancho1) = {round(h_a1, 3)}')

print(f'\nH(largo1|ancho1) = {round(h_cond_l_a_1, 3)}')
print(f'H(largo1) = {round(h_l1, 3)}')

print('\n------- Clase 2 --------')
h_cond_l_a_2 = h_cond(conj_c2_norm)
print(f'H(ancho2|largo2) = {round(h_cond_a_l_2, 3)}')
print(f'H(ancho2) = {round(h_a2, 3)}')

print(f'\nH(largo2|ancho2) = {round(h_cond_l_a_2, 3)}')
print(f'H(largo2) = {round(h_l2, 3)}')

------- Clase 1 --------
H(ancho1|largo1) = 1.248
H(ancho1) = 4.511

H(largo1|ancho1) = 1.22
H(largo1) = 5.358

------- Clase 2 --------
H(ancho2|largo2) = 1.125
H(ancho2) = 4.224

H(largo2|ancho2) = 1.063
H(largo2) = 5.161


En ambos casos, la entropía condicional no es muy cercana a la individual, es decir, el largo y ancho de una misma clase se aportan algo de información entre sí. Esto implica que no pueden ser independientes las características. 

Luego, se prueba con los anchos y largos entre clases.

In [7]:
print('\n------- Anchos ---------')
print(f'H(ancho1|ancho2) = {round(h_cond_a, 3)}')
print(f'H(ancho1) = {round(h_a1, 3)}')

conj_ancho2, conj_ancho_norm2 = conjunta(c2['ancho'], c1['ancho'], 5)
h_cond_a2 = h_cond(conj_ancho_norm2)
print(f'\nH(ancho2|ancho1) = {round(h_cond_a2, 3)}')
print(f'H(ancho2) = {round(h_a2, 3)}')

print('\n------- Largos ---------')
print(f'H(largo1|largo2) = {round(h_cond_l, 3)}')
print(f'H(largo1) = {round(h_l1, 3)}')

conj_largo2, conj_largo_norm2 = conjunta(c2['largo'], c1['largo'], 5)
h_cond_l2 = h_cond(conj_largo_norm2)
print(f'\nH(largo2|largo1) = {round(h_cond_l2, 3)}')
print(f'H(largo2) = {round(h_l2, 3)}')


------- Anchos ---------
H(ancho1|ancho2) = 1.847
H(ancho1) = 4.511

H(ancho2|ancho1) = 1.711
H(ancho2) = 4.224

------- Largos ---------
H(largo1|largo2) = 1.852
H(largo1) = 5.358

H(largo2|largo1) = 1.683
H(largo2) = 5.161


En este caso, hay diferencia entre las entropías, pero es de menor magnitud que la anterior. Para ambas características es menor a $0.1$ bits. Entre clases, el aporte de las variables que condicionan a la descripición es poco, por lo que las distribuciones de las características son diferentes. Este cálculo sugiere que no se debería rechazar la independencia entre clases.

### Entropía relativa
La entropía relativa $D(p||q)$, también conocida como Kullback Leibler, es definida en el libro como la divergencia entre dos funciones de probabilidad. Esta medida es siempre positiva y es nula si y solo sí las funciones de probabilidad son iguales. Se calcula de la siguiente manera:
$$
D(p(x)||q(x)) = \sum_{x \in X} p(x) \log_2{\frac{p(x)}{q(x)}} = E_p(log_2(\frac{p(x)}{q(x)})),
$$
donde se cumple que si $q(x) = 0$, entonces $p(x)\log_2{\frac{p(x)}{0} = \inf}$, y, si $p(x) = 0$, entonces $0\log_2{\frac{0}{q(x)} = 0}$.

La entropía relativa sirve como medida de la ineficiencia generada por el uso de la función de probabilidad $q(x)$ para la variable aleatoria $x$, cuando su función de distribución, verdaderamente, es $p(x)$. Si se trabaja con bits, la entropía relativa sería la cantidad de bits adicionales necesarios para describir $x$ con $q(x)$, en vez de usar $p(x)$. Sería utilizar $H(x) + D(p||q)$ bits en vez de $H(x)$ bits.

De manera parecida, se define la entropía relativa entre dos probabilidades conjuntas, $p(x, y)$ y $q(x, y)$:
$$
D(p(x, y)||q(x, y)) = \sum_{x \in X}\sum_{y \in Y} p(x, y) \log_2{\frac{p(x, y)}{q(x, y)}},
$$

Se encuentra la entropía relativa entre clases.

In [8]:
def h_relat_conj(conj1, conj2, r, c):
    """Cálculo de entropía relativa de dos probabilidades conjuntas. """

    d = 0
    for i in range(r):
        for j in range(c):
            if conj1[i][j]!=0 and conj2[i][j]!=0:
                d += conj1[i][j]*(log2(conj1[i][j]/conj2[i][j]))
    return d


h_relat_clases12 = h_relat_conj(conj_c1_norm, conj_c2_norm, 5, 5)
h_relat_clases21 = h_relat_conj(conj_c2_norm, conj_c1_norm, 5, 5)

print('------- Clase 1 --------')
print(f"D(p(largo1, ancho1)||q(largo2, ancho2)) = {round(h_relat_clases12, 3)}")
print(f"D(q(largo2, ancho2)||p(largo1, ancho1)) = {round(h_relat_clases21, 3)}")


------- Clase 1 --------
D(p(largo1, ancho1)||q(largo2, ancho2)) = 0.327
D(q(largo2, ancho2)||p(largo1, ancho1)) = 0.392


Por las entropías relativas calculadas, se puede decir que hay una ineficiencia en usar la distribución de la clase 1 para describir a la clase 2, y lo mismo al revés. Lo cual implica que las clases son diferentes. La independencia entre variables de distintas clases es lógica.

#### Información mutua
A partir de la entropía relativa, se define la información mutua entre dos variables. Esta es una medida de la cantidad de información que una variable contiene de otra. En específico, si se tiene dos variables aleatorias $X$ e $Y$, con función de probabilidad conjunta $p(x, y)$ y marginales $p(x)$ y $p(y)$, se define la información mutua como:
$$
I(X, Y) = D(p(x, y)||p(x)p(y)) = \sum_{x \in X}\sum_{y \in Y} p(x, y) \log_2{\frac{p(x,y)}{p(x)p(y)}}
$$
Por la definición de entropía relativa, más chico el número que de la información mutua, más parecidas son las distribuciones $p(x, y)$ y $p(x)p(y)$, es decir, menos información contiene $X$ de $Y$ y al revés. En el caso de que sean iguales, por ende son independientes las variables, la información mutua sería $I(X, Y) = 0$. 

In [9]:
def inf_mutua(conj, marg1, marg2, anch1, anch2):
    """Cálculo de información mutua. """
    
    h = 0
    for i in range(anch1):
        for j in range(anch2):
            if conj[i][j]!=0:
                h += conj[i][j]*(log2(conj[i][j]/(marg1[i]*marg2[j])))

    return h

inf_mutua_c1 = inf_mutua(conj_c1_norm, marg_l1, marg_a1, 5, 5)
inf_mutua_c2 = inf_mutua(conj_c2_norm, marg_l2, marg_a2, 5, 5)
inf_mutua_ancho = inf_mutua(conj_ancho_norm, marg_a1, marg_a2, 5, 5)
inf_mutua_largo = inf_mutua(conj_largo_norm, marg_l1, marg_l2, 5, 5)

print('------- Clase 1 --------')
print(f"I(largo1, ancho1) = {round(inf_mutua_c1, 3)}")

print('\n------- Clase 2 --------')
print(f"I(largo2, ancho2) = {round(inf_mutua_c2, 3)}")

print('\n------- Anchos ---------')
print(f"I(ancho1, ancho2) = {round(inf_mutua_ancho, 3)}")

print('\n------- Largos ---------')
print(f"I(largo1, largo2) = {round(inf_mutua_largo, 3)}")

------- Clase 1 --------
I(largo1, ancho1) = 0.678

------- Clase 2 --------
I(largo2, ancho2) = 0.666

------- Anchos ---------
I(ancho1, ancho2) = 0.08

------- Largos ---------
I(largo1, largo2) = 0.046


Observando los resultados, se puede notar que la información mutua entre el largo y ancho de una clase, que se esperaría tengan dependencia por lo visto anteriormente, es lejano a cero en comparación a la información mutua entre los ancho o largos. La cercanía a cero implica la independencia de las variables, porque el producto de sus marginales es muy parecido a su probabilidad conjunta. Se refuerza que entre caracterísitcas de una misma clase hay dependencia, pero entre clases hay independencia.

Otra manera de definir la información mutua es a partir de la marginal y condicional de las variables.
\begin{equation}
\begin{aligned}
I(X, Y) = H(X) - H(X|Y) = H(Y) - H(Y|X)
\end{aligned} \tag{4}
\end{equation}

Con los datos, se verifica esta igualdad:

In [10]:
print('------- Clase 1 --------')
print(f"I(largo1, ancho1) = {round(inf_mutua_c1, 3)}")
print(f"H(ancho1) - H(ancho1|largo1) = {round(h_a1 - h_cond_a_l_1, 3)}")

print('\n------- Clase 2 --------')
print(f"I(largo2, ancho2) = {round(inf_mutua_c2, 3)}")
print(f"H(ancho2) - H(ancho2|largo2) = {round(h_a2 - h_cond_a_l_2, 3)}")

print('\n------- Anchos ---------')
print(f"I(ancho1, ancho2) = {round(inf_mutua_ancho, 3)}")
print(f"H(ancho1) - H(ancho1|ancho2) = {round(h_a1 - h_cond_a, 3)}")

print('\n------- Largos ---------')
print(f"I(largo1, largo2) = {round(inf_mutua_largo, 3)}")
print(f"H(largo1) - H(largo1|largo2) = {round(h_l1 - h_cond_l, 3)}")

------- Clase 1 --------
I(largo1, ancho1) = 0.678
H(ancho1) - H(ancho1|largo1) = 3.263

------- Clase 2 --------
I(largo2, ancho2) = 0.666
H(ancho2) - H(ancho2|largo2) = 3.099

------- Anchos ---------
I(ancho1, ancho2) = 0.08
H(ancho1) - H(ancho1|ancho2) = 2.665

------- Largos ---------
I(largo1, largo2) = 0.046
H(largo1) - H(largo1|largo2) = 3.505


#### Conclusión
En el trabajo se comprende que las variables dentro de una clase son dependientes, pero entre clases son independientes. Esto implica que el largo y ancho de una hoja están relacionados, por lo que conocer una de estas características te dá información sobre la otra. En cambio, entre clases no hay una relación de dependencia. Saber la distribución del largo de una clase no aporta a comprender la distribución de la otra. Ambas conclusiones se comprendieron a partir de resultados que se esperaría obtener de las ditintas entropías en el caso que sean independientes o no. 